In [5]:
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

options = webdriver.ChromeOptions()
options.set_capability('goog:loggingPrefs', {"performance": "ALL", "browser": "ALL"})
# Adding user agent to look more like a real browser
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.set_page_load_timeout(30)

In [4]:
url = "https://www.sofascore.com/football/match/manchester-united-leicester-city/GK#id:12437027,tab:statistics"

In [3]:
def get_score(url,match_id, driver):
    # Clear existing logs
    #driver.get("about:blank")
    #logs = driver.get_log("performance")
    
    # Load the page
    """
    print(f"Loading URL: {url}")
    driver.get(url)
    """
    
    """
    # Wait for the page to load properly
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "body"))
        )
    except:
        print("Timeout waiting for page to load")
    """
    """
    # Interact with the page to trigger API calls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
    time.sleep(0.1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.2)

    soup = BeautifulSoup (driver.page_source, "html.parser")

    meta_data = soup.find_all('script', type = 'application/ld+json')

    meta_data = json.loads(meta_data[1].string)
    """
    # Direct API approach - try to call the API directly
    
    if match_id:
        print(f"Extracted match ID: {match_id}")
        urls = []
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/statistics")
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}")
        urls.append(f"https://www.sofascore.com/api/v1/event/{match_id}/lineups")

        
        data = {}
        for i in range(0,len(urls)):
            api_url = urls[i]
            # Open the API URL directly
            #driver.execute_script(f"window.open('{api_url}', '_blank');")

            driver.get(api_url)
            #time.sleep(2)
            
            # Switch to the new tab
            #driver.switch_to.window(driver.window_handles[-1])
            #time.sleep(1)
            
            # Get the page source which should contain the JSON
            page_source = driver.page_source
            
            # Extract JSON from the page source
            if "application/json" in page_source or "{" in page_source:
                start_idx = page_source.find("{")
                end_idx = page_source.rfind("}") + 1
                if start_idx >= 0 and end_idx > start_idx:
                    json_str = page_source[start_idx:end_idx]
                    try:
                        data[i] = json.loads(json_str)
                    except json.JSONDecodeError:
                        print("Failed to parse JSON response")
            
            # Close the tab and switch back
            #driver.close()
            #driver.switch_to.window(driver.window_handles[0])

        #data[len(urls)] = meta_data
        return data

In [7]:
data =  (get_score(url,12437027,driver))

Extracted match ID: 12437027


In [33]:
def parse_data(data):
    ## statisitics = 0 
    ## event_data = 1
    ## formation = 2
    
    statistics = data[0]['statistics'][0]['groups']
    event = data[1]['event']
    lineups = data[2]
    
    match_data = {}
    
    ## get stats data from match 
    stats_headers = ['Ball possession','Expected goals','Big chances', 'Total shots', 'Goalkeeper saves','Corner kicks','Fouls','Passes','Tackles','Free kicks','Yellow cards','Red cards']
    stats_shots_headers = ['Total shots', 'Shots on target','Hit woodwork','Shots off target','Blocked shots', 'Shots inside box','Shots outside box']
    stats_attack_headers = ['Big chances missed','Through balls','Touches in penalty area','Fouled in final third', 'Offsides']
    stats_passes_headers = ['Accurate passes','Throw-ins','Final third entries','Long balls', 'crosses']
    stats_duels_headers = ['Duels','Dispossessed','Ground duels','Aerial duels','Dribbles']
    stats_defending_headers = ['Total tackles', 'Interceptions','Recoveries','Clearances','Errors lead to a shot']
    stats_goalkeeper_headers = ['Total saves','Goals prevented','Big saves','High claims','Goal kicks']

    all_headers = (
        stats_headers +
        stats_shots_headers +
        stats_attack_headers +
        stats_passes_headers +
        stats_duels_headers +
        stats_defending_headers +
        stats_goalkeeper_headers
    )
    match_data.update({header: "0" for header in all_headers})
    for home
    # parse stats API 
    for group in statistics:
        for stat in group['statisticsItems']:
            if stat['name'] in all_headers:
                match_data[stat['name']] = stat['awayValue']
    print (match_data)

In [34]:
parse_data(data)

{'teamName': 'poo', 'Ball possession': 46, 'Expected goals': 0.91, 'Big chances': 0, 'Total shots': 18, 'Goalkeeper saves': 3, 'Corner kicks': 4, 'Fouls': 6, 'Passes': 461, 'Tackles': 20, 'Free kicks': 6, 'Yellow cards': 0, 'Red cards': '0', 'Shots on target': 5, 'Hit woodwork': 1, 'Shots off target': 8, 'Blocked shots': 5, 'Shots inside box': 7, 'Shots outside box': 11, 'Big chances missed': 0, 'Through balls': 1, 'Touches in penalty area': 17, 'Fouled in final third': 1, 'Offsides': 2, 'Accurate passes': 359, 'Throw-ins': 25, 'Final third entries': 33, 'Long balls': 31, 'crosses': '0', 'Duels': 52, 'Dispossessed': 11, 'Ground duels': 32, 'Aerial duels': 24, 'Dribbles': 6, 'Total tackles': 20, 'Interceptions': 9, 'Recoveries': 59, 'Clearances': 31, 'Errors lead to a shot': 0, 'Total saves': 3, 'Goals prevented': 0.3835, 'Big saves': 1, 'High claims': 1, 'Goal kicks': 8}
